<a href="https://colab.research.google.com/github/TokusimaHonda/google_colaboratory/blob/master/%E7%94%BB%E5%83%8F%E3%82%B9%E3%82%AF%E3%83%AC%E3%82%A4%E3%83%94%E3%83%B3%E3%82%B0_Post.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import urllib.request, urllib.error, urllib.parse
import requests
import os
from google.colab import drive

import json

## 検索ワード指定

In [ ]:
search_word = '検索したい画像'
max_page = 1 # ページ数（20枚/ページ）

## Googleドライブマウント設定関数

In [ ]:
#Googleドライブマウントして、フォルダ作成
def google_drive_mount(save_dir_path):
  drive.mount('./gdrive')
  # ディレクトリが存在しない場合は新規作成
  os.makedirs(save_dir_path, exist_ok=True)

## 画像ダウンロード用関数

In [ ]:
def download_image(url, file_path):
  r = requests.get(url, stream=True)

  if r.status_code == 200:
    with open(file_path, "wb") as f:
      f.write(r.content)

## 画像line投稿関数

In [ ]:
def chat_post_image(yahoo_url, file_path,chat_type):
  

  #Yahoo画像取得
  requests_yahoo = requests.get(yahoo_url, stream=True)

  if requests_yahoo.status_code == 200:

    if chat_type == "line":
      #画像を保存
      with open(file_path, "wb") as f:
        f.write(requests_yahoo.content)

      #lineトークン設定
      line_url = "https://notify-api.line.me/api/notify"
      access_token = 'line_token'
      headers = {'Authorization': 'Bearer ' + access_token}
      payload = {'message': file_path}

      #画像をLineに投稿
      files = {'imageFile': open(file_path, 'rb')}
      requests_line = requests.post(line_url, headers=headers, params=payload, files=files,)
      
      #画像を保存したくない場合はコメントアウトする
      os.remove(file_path)

    elif chat_type == "slack":
      #slackurl設定
      slack_url = "slack_url"
      post_json = {
            "text": "テキスト",
            "attachments": [{
                "fields": [
                  {
                        "title": "タイトル",
                        "value": "説明",
                  }],
            "image_url": yahoo_url
            }]
      }
      print(yahoo_url)
      requests_slack = requests.post(slack_url, data = json.dumps(post_json))
  




## Yahoo画像検索スクレイピング関数

In [ ]:
def download_yahoo_search_image(keyword,chat_name):
  img_cnt = 1
  download_cnt = 1
  for i in range(max_page):
    #検索文字列をエンコードして、Yahoo画像検索用URL作成
    url = 'https://search.yahoo.co.jp/image/search?p={}&ei=UTF-8&b={}'.format(urllib.parse.quote(keyword), img_cnt)

    #Yahoo画像検索ページを開き、要素取得
    req = requests.get(url)
    soup = BeautifulSoup(req.text, features="html.parser")
    
    #画像検索結果のimgタグを取得
    div = soup.find('div', id='gridlist')
    img_tags = div.find_all('img')
    
    for img_tag in img_tags:
      #画像URLのみを取得
      img_url = img_tag.get("src")
      #print(img_url)
      #ファイル名・保存先パスを作成
      file_name = str(download_cnt) + '_' + '.jpg'
      image_path = os.path.join(google_drive_save_dir, file_name)
      
      #画像を保存
      #download_image(url=img_url, file_path=image_path)
      chat_post_image(yahoo_url=img_url, file_path=image_path,chat_type=chat_name)
      download_cnt += 1
    
    #次のページ
    img_cnt += 20    
    

## スクレイピング処理

In [ ]:
#Googleドライブ保存フォルダ作成
google_drive_save_dir = "./gdrive/My Drive/Colab Notebooks/{}".format(search_word)
google_drive_mount(google_drive_save_dir)
download_yahoo_search_image(search_word,'slack')
